In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Data Augmentation and Normalization
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-100 dataset
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define CNN Model (ResNet-like architecture)
class CNNModel(nn.Module):
    def __init__(self, num_classes=100):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.batch_norm4 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.batch_norm1(self.conv1(x))))
        x = self.pool(F.relu(self.batch_norm2(self.conv2(x))))
        x = self.pool(F.relu(self.batch_norm3(self.conv3(x))))
        x = self.pool(F.relu(self.batch_norm4(self.conv4(x))))
        
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Initialize Model
model = CNNModel().to(device)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
def train_model(model, trainloader, testloader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_acc = 100. * correct / total
        
        # Validation
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        test_acc = 100. * correct / total
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}, "
              f"Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%")

# Train the model
train_model(model, trainloader, testloader, epochs=20)

# Save Model
torch.save(model.state_dict(), "cifar100_cnn.pth")
print("Model saved as cifar100_cnn.pth")


100%|██████████████████████████████████████████████████████████████| 169001437/169001437 [00:02<00:00, 72961090.20it/s]


Extracting ./data\cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/20, Loss: 3.9818, Train Acc: 8.12%, Test Acc: 14.06%
Epoch 2/20, Loss: 3.4701, Train Acc: 15.71%, Test Acc: 22.87%
Epoch 3/20, Loss: 3.1249, Train Acc: 21.59%, Test Acc: 28.64%
Epoch 4/20, Loss: 2.8917, Train Acc: 26.30%, Test Acc: 32.75%
Epoch 5/20, Loss: 2.6955, Train Acc: 30.26%, Test Acc: 36.98%
Epoch 6/20, Loss: 2.5383, Train Acc: 33.32%, Test Acc: 41.38%
Epoch 7/20, Loss: 2.4006, Train Acc: 36.16%, Test Acc: 40.96%
Epoch 8/20, Loss: 2.2864, Train Acc: 38.94%, Test Acc: 44.65%
Epoch 9/20, Loss: 2.1871, Train Acc: 41.13%, Test Acc: 46.12%
Epoch 10/20, Loss: 2.1007, Train Acc: 43.17%, Test Acc: 48.87%
Epoch 11/20, Loss: 2.0135, Train Acc: 45.11%, Test Acc: 48.20%
Epoch 12/20, Loss: 1.9510, Train Acc: 46.66%, Test Acc: 50.88%
Epoch 13/20, Loss: 1.8902, Train Acc: 47.89%, Test Acc: 50.20%
Epoch 14/20, Loss: 1.8233, Train Acc: 49.53%, Test Acc: 52.92%
Epoch 15/20, Loss: 1.7712, Train Acc: 

In [3]:
model.eval()

correct = 0
total = 0

for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)

    predictions = model(images)
    predictions = torch.argmax(predictions, dim=1)

    correct_predictions = (predictions == labels).sum().item()
    correct += correct_predictions
    total += labels.shape[0]

print(correct)
print(total)

5695
10000
